In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys

data_dir = os.path.join(os.environ["PROJECT_ROOT"], "data")

In [ ]:
import geemap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import src.data_preprocessing.create_aux_data as cad
import src.data_preprocessing.data_utils as du
import src.data_preprocessing.gee_utils as gu

In [ ]:
# filename='s2bms_bioclim_lc_data.csv'

# ## Load auxiliary data:
# if data_dir is None:
#     data_dir = os.path.join(os.environ['PROJECT_ROOT'], 'data')
# path_butterfly_aux_target = os.path.join(data_dir, 'source', 'butterflies', filename)
# assert os.path.exists(
#     path_butterfly_aux_target
# ), f"Butterfly auxiliary data file does not exist: {path_butterfly_aux_target}"
# df_bioclim_lc = pd.read_csv(path_butterfly_aux_target)
# corine_keys = [k for k in df_bioclim_lc.iloc[0].index if 'corine_frac_' in k]


# target_cols = [c for c in df_bioclim_lc.columns if ('corine' in c and 'top' not in c)]
# target_cols.sort()
# sub_np = df_bioclim_lc[target_cols].to_numpy()

# # reproducible randomness
# rng = np.random.default_rng(seed=42)
# noise = rng.uniform(0, 1e-8, size=sub_np.shape)
# sub_np_noisy = sub_np + noise

# # top-5
# top_k_idx = np.argsort(sub_np_noisy, axis=1)[:, -5:][:, ::-1]
# target_cols_np = np.array(target_cols)

# for i in range(5):
#     df_bioclim_lc[f"aux_corine_frac_top_{i + 1}"] = target_cols_np[top_k_idx[:, i]]
# df_bioclim_lc.to_csv(path_butterfly_aux_target, index=False)

In [ ]:
df_merged = cad.create_butterfly_aux_data(
    download_aux_data=False, prefix_aux="aux_", prefix_target="target_", save_file=True
)
df_merged.head()

In [ ]:
tmp = gu.get_gee_image_from_coord(
    (df_merged.iloc[0].lon, df_merged.iloc[0].lat),
    collection_name="corine",
    patch_size=1400,
    threshold_size=128,
)
tmp.getInfo()["bands"]
# tmp = gu.download_gee_image((df_merged.iloc[0].lon, df_merged.iloc[0].lat), name='test', path_save='/Users/tplas/Downloads',
#                             collection_name='corine', verbose=0, patch_size=1400, year=2017)

In [ ]:
# coord_list = [(df_merged.iloc[i].lon, df_merged.iloc[i].lat) for i in range(10)]
# name_list = [df_merged.iloc[i].name_loc for i in range(10)]
df_s2bms_presence = du.load_s2bms_presence()
coord_list = df_s2bms_presence.tuple_coords.tolist()
name_list = df_s2bms_presence.name_loc.tolist()

tmp = gu.download_list_coord(
    coord_list,
    name_list=name_list,
    path_save="/Users/tplas/data/s2bms_butterfly_sent2-4band_256/",
    name_group="s2bms",
    list_collections=["sentinel2"],
    pixel_patch_size=256,
    start_index=0,
)

In [ ]:
import copy

im_failed = copy.deepcopy(tmp)

In [ ]:
pd.read_csv("../data/model_ready/s2bms_presence_with_aux_data.csv").head()

In [ ]:
df_s2bms_presence = du.load_s2bms_presence()
df_s2bms_presence.tuple_coords